__NOTE: Do NOT 'Run All' on this notebook.__
- Edit the variables in the cell below.

In [ ]:
#You _will_ need to edit this variable
jobName='Debra'

#You may need to edit these (if you know what you're doing.)
enableBucket=False
maxResolution='768,768'
xformers=True

- Continue running each cell one at a time.

In [ ]:
imageDir='/workspace/training'
configFile='/workspace/runpod_ai/scripts/LoraBasicSettings.json'

In [ ]:
!mkdir -p $imageDir
!mkdir -p $imageDir/logs
!mkdir -p $imageDir/images/tmp
!mkdir -p $imageDir/output
!mkdir -p $imageDir/reg
print('Upload training pictues to '+imageDir+'/images/tmp')

__Pause here__
- Upload your training images to the directory named above.
- Continue running cells (one at a time) when all pictures are uploaded.

In [ ]:
import os

directory = imageDir+'/images/tmp'  # Specify the directory path
numFiles = len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])

# min 100/image, min 1500/epoch
numSteps = 1500/numFiles
if numSteps < 100:
    numSteps = 100
print('Steps per image:  '+str(numSteps))
print('Steps per epoch:  '+str(numSteps*numFiles))

newDirName=str(numSteps)+'_'+jobName
%cd $imageDir/images
!mv tmp $newDirName
print('Folder to caption:  '+imageDir+'/images/'+newDirName)

__Pause here__
- Run 'Utilities/WD14 Captioning'
- Continue running cells (one at a time) when captioning is finished.

In [ ]:
origConfigFile=configFile+'.orig'
!cp $origConfigFile $configFile
!sed -i 's/\"output_name\": \"latest\",/\"output_name\": \"{jobName}\",/' $configFile
!sed -i 's/\"max_resolution\": \"768,768\",/\"max_resolution\": \"{maxResolution}\",/' $configFile
enableBucketStr=str(enableBucket).lower()
!sed -i 's/\"enable_bucket\": true,/\"enable_bucket\": {enableBucketStr},/' $configFile
xformersStr=str(xformers).lower()
!sed -i 's/\"xformers\": false,/\"xformers\": {xformersStr},/' $configFile
print('Configuration file:  '+configFile)

__Almost done.__
- Load config file in 'Dreambooth LoRA'
- Set up sampling in 'Dreambooth LoRA/Sample images config'
- Start training in kohya_ss UI.